In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from plotnine import *
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [6]:
# load in pre-cleaned deaths data for Texas and comparison states
deaths = pd.read_csv(r"C:\Users\annap\Downloads\deaths_tx.csv")


In [7]:
# two separate dfs - one for texas and one for comp states

texas = deaths[deaths["StateName"] == "Texas"]
comp = deaths[deaths["StateName"] != "Texas"]

### Helper functions 

In [8]:
# function to select only overdose records
# double check this

def select_overdose(record):
    """Simple function to select only overdose records"""

    if record == "All other non-drug and non-alcohol causes":
        return 0

    if record == "All other alcohol-induced causes":
        return 0

    if record == "All other drug-induced causes":
        return 0

    if record == "Alcohol poisonings (overdose) (X45, X65, Y15)":
        return 0

    if record == "Drug poisonings (overdose) Unintentional (X40-X44)":
        return 1

    if record == "Drug poisonings (overdose) Suicide (X60-X64)":
        return 1

    if record == "Drug poisonings (overdose) Undetermined (Y10-Y14)":
        return 1

    else:
        return "error"
    

# copy to fix the dreaded "A value is trying to be set on a copy of a slice" error
tx_deaths = texas.copy()
comp_deaths = comp.copy()

# apply new function to our df
tx_deaths["overdose"] = tx_deaths["Drug/Alcohol Induced Cause"].apply(lambda x: select_overdose(x))
comp_deaths["overdose"] = comp_deaths["Drug/Alcohol Induced Cause"].apply(lambda x: select_overdose(x))

assert len(tx_deaths[tx_deaths["overdose"] == "error"]) == 0
assert len(comp_deaths[comp_deaths["overdose"] == "error"]) == 0

# filter accordingly based on new column
tx_deaths = tx_deaths[tx_deaths["overdose"] != 0]
comp_deaths = comp_deaths[comp_deaths["overdose"] != 0]

In [9]:
# calculate overdoses per 100_000 residents

tx_deaths["overdose_per_100k"] = tx_deaths["Deaths"] / tx_deaths["Population"] * 100_000
comp_deaths["overdose_per_100k"] = comp_deaths["Deaths"] / comp_deaths["Population"] * 100_000

In [10]:
# groupby year and county
tx_result = tx_deaths.groupby(["Year", "County"])["overdose_per_100k"].sum().reset_index()
comp_result = comp_deaths.groupby(["Year", "StateName", "CountyName"])["overdose_per_100k"].sum().reset_index()

In [11]:
tx_result

,Year,County,overdose_per_100k
0,2004.0,Anderson,19.557294
1,2004.0,Bexar,8.461483
2,2004.0,Brazoria,7.088203
3,2004.0,Cameron,4.368016
4,2004.0,Collin,6.474566
...,...,...,...
221,2010.0,Tom Green,18.070439
222,2010.0,Travis,11.450719
223,2010.0,Webb,9.149859
224,2010.0,Wichita,12.140158


In [12]:
# calculate summary stats
tx_res = pd.DataFrame(tx_result.describe()["overdose_per_100k"]).rename(columns={"overdose_per_100k": "Overdoses per 100k Residents - Texas"})
comp_res = pd.DataFrame(comp_result.describe()["overdose_per_100k"]).rename(columns={"overdose_per_100k": "Overdoses per 100k Residents - Comp States"})

In [13]:
stats = pd.concat([tx_res, comp_res], axis=1)
stats

,Overdoses per 100k Residents - Texas,Overdoses per 100k Residents - Comp States
count,226.000000,301.000000
mean,10.471761,11.017577
std,6.469153,4.808954
min,1.438121,3.163785
25%,6.541813,7.680168
50%,9.107748,10.058255
75%,11.957830,13.130388
max,42.744176,39.497591


In [14]:
# overwrite tx result to be grouped by year only
tx_result = tx_result.groupby("Year")["overdose_per_100k"].mean().reset_index()
comp_result = comp_result.groupby(["Year"])["overdose_per_100k"].mean().reset_index()

In [15]:
tx_res_checkpoint = tx_result.copy()
comp_res_checkpoint = comp_result.copy()


# assert length is the same
assert (len(tx_result) == len(tx_res_checkpoint)) & (len(comp_result) == len(comp_res_checkpoint))

## Pre-post plot - overdoses per 100k through the years

In [16]:
# split into before 2007 and after 2007

tx_b4 = tx_result[tx_result["Year"] < 2007]
tx_after = tx_result[tx_result["Year"] >= 2007]


#tx_after = tx_after[tx_after["Year"] != 2007]

In [17]:
comp_b4 = comp_result[comp_result["Year"] < 2007]
comp_after = comp_result[comp_result["Year"] >= 2007]


In [18]:
x = "Year"
y = "overdose_per_100k"

In [25]:
# Function to plot a vertical line at year of policy implementation
def vertical_line(year):
   
    line = alt.Chart(pd.DataFrame({
    'Year': [year],
    'color': ["black"]
    })).mark_rule().encode(
    x='Year:Q',
    color=alt.Color('color:N', scale=None)
    )
 
    return line


In [26]:
# creating the vertical line at 2007, the year of implementation of policy changes in Texas
line = vertical_line(2007)



In [27]:
# creating the regression model and calculating the error bands for creating the plots
 
def get_reg_fit_and_ci(data, color, xvar, yvar, legend, alpha=0.05):
   
    # Creating the grid for predicted values
    colour = color
    years = [2004,2005,2006,2007,2008,2009,2010]
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})
 
    # Fitting the model and making the predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)
   
    # Utilizing the predictions to create the points and error bands in the chart
    predictions["Before/After"] = f"{legend}"
    reg = (
        alt.Chart(predictions)
        .mark_line()
        .encode(
            x=xvar,
            y=alt.Y(yvar),
            color = alt.Color("Before/After",legend=alt.Legend(title = "Before/After Policy Change"))
        )
    )
 
    ci = (
        alt.Chart(predictions)
        .mark_errorband()
        .encode(
            alt.X(f"{xvar}:Q", axis=alt.Axis(format=".0f", values=years)),
            y=alt.Y(
                "ci_low",
                title="Opioid Overdose Deaths Per 100K Residents",
                scale=alt.Scale(zero=False),
            ),
            y2="ci_high",
            color=alt.value(f"{color}"),
        )
    )
    chart = ci + reg
    return predictions, chart
 


In [28]:
# Using the get_reg_fit_and_ci function to create the charts
 
def build_chart(data, color, xvar, yvar, legend, alpha=0.05):
    fit, reg_chart = get_reg_fit_and_ci(
        data=data, color=color, xvar=xvar, yvar=yvar,  legend=legend, alpha=alpha,
    )
    return reg_chart


In [31]:
# creating the final plot for pre-post analysis of drug overdose deaths in Texas
 
texas_before_chart = build_chart(
    tx_b4, "orange", "Year","overdose_per_100k", "Texas Before",  alpha=0.05
)
 
texas_after_chart = build_chart(
    tx_after, "blue", "Year", "overdose_per_100k", "Texas After", alpha=0.05
)
 
texas_final_pre_post_deaths = (texas_before_chart + texas_after_chart + line).properties(title="Pre Post Analysis Of Opioid Overdose Deaths In Texas")
 
texas_final_pre_post_deaths
 


alt.LayerChart(...)

In [24]:
(reg_chart_tx_b4 + reg_chart_tx_after + reg_chart_comp_b4 + reg_chart_comp_after + line).properties(width=700, height=500)

alt.LayerChart(...)